# How to engineer the joint angles for a new behaviour

In this notebook we will take the first steps to implement the lunging behavior. This behaviour is quite distant from the behaviours we have seen so far in the course though with some creativity it is relatively easy to engineer the time series leading to that behaviour.  

An example of joint angles engineering for grooming can be found at **week5/Extract_other_modules.ipynb**.


Example of lunging behaviour:

<p float="left">
    <video align="center" width="700" controls>
    <source src="source/lunging.mp4" type="video/mp4">
    </video>
</p>
Credits: Hoyer SC, Eckart A, Herrel A, Zars T, Fischer SA, Hardie SL, Heisenberg M (2008). Octopamine in male aggression of Drosophila. Current Biology 18: 159-167

# Selecting a starting behaviour

to start with we have to select a starting points. This starting point will give us the required cyclical behaviour of joints. Once this is done, we can scale the joint angles, offset them or even apply them to a different joint to get the desired motion. For the lunging behaviour it seems quite clear that the middle legs are getting extended in a classical way while the hindlegs are getting retracted. The best behaviour to start from is walking. Note that depending on your behaviour, it might be easier to start from the fly resting pose, from grooming or from the walking behaviour. Then the joints can be offset, inverted, scaled or replaced by a sine wave among other possibilities.

For this exact behaviour, we propose to keep adhesion on in the hind and middle leg and then go through the stance for the middle and through the swing for the hindlegs. This will pulling them closer to each other. In a next step, we can extend further the middle legs so that the fly lunges.

In [1]:
from flygym.mujoco import Parameters, NeuroMechFly

from flygym.mujoco.examples.common import PreprogrammedSteps
import numpy as np

import matplotlib.pyplot as plt

In [2]:
nmf = NeuroMechFly(Parameters(enable_adhesion=True, draw_adhesion=True))
preprogrammed_steps = PreprogrammedSteps()

swing_periods = preprogrammed_steps.swing_period

legs = preprogrammed_steps.legs

standing_action = []
for leg in legs:
    if leg.endswith("M"):
        standing_action.extend(
            preprogrammed_steps.get_joint_angles(leg, swing_periods[leg][1])
        )
    else:
        standing_action.extend(preprogrammed_steps.get_joint_angles(leg, 0.0))

stand_action = {"joints": standing_action, "adhesion": np.zeros(len(legs))}

# Let the fly stand on the floor first
for i in range(int(0.2 // nmf.timestep)):
    nmf.step(stand_action)


run_time = 0.3
target_num_steps = int(run_time // nmf.timestep)

foreleg_ids = np.zeros(target_num_steps)
middle_stance_ids = np.linspace(swing_periods["RM"][1], 2 * np.pi, target_num_steps)
hind_swing_ids = np.linspace(0.0, swing_periods["RH"][1], target_num_steps)

adhesion_action = np.array([0.0 if leg.endswith("F") else 1.0 for leg in legs])

all_joint_angles = []

for i in range(target_num_steps):
    joint_angles = []
    for leg in legs:
        if leg.endswith("F"):
            joint_angles.extend(
                preprogrammed_steps.get_joint_angles(leg, foreleg_ids[i])
            )
        elif leg.endswith("M"):
            joint_angles.extend(
                preprogrammed_steps.get_joint_angles(leg, middle_stance_ids[i])
            )
        else:
            joint_angles.extend(
                preprogrammed_steps.get_joint_angles(leg, hind_swing_ids[i])
            )

    all_joint_angles.append(joint_angles.copy())

    action = {"joints": np.array(joint_angles), "adhesion": adhesion_action}
    nmf.step(action)
    nmf.render()

nmf.save_video("lunging_base.mp4", 0)

Now that we are there, it is clear that we need to fully extend the midlegs. In the "zero pose" (joint angles == 0.0), the midlegs are fully extended.

In [3]:
nmf = NeuroMechFly(Parameters(enable_adhesion=True, draw_adhesion=True))
preprogrammed_steps = PreprogrammedSteps()

swing_periods = preprogrammed_steps.swing_period

legs = preprogrammed_steps.legs

standing_action = []
for leg in legs:
    if leg.endswith("M"):
        standing_action.extend(
            preprogrammed_steps.get_joint_angles(leg, swing_periods[leg][1])
        )
    else:
        standing_action.extend(preprogrammed_steps.get_joint_angles(leg, 0.0))

stand_action = {"joints": standing_action, "adhesion": np.zeros(len(legs))}

# Let the fly stand on the floor first
for i in range(int(0.2 // nmf.timestep)):
    nmf.step(stand_action)


run_time = 0.3
target_num_steps = int(run_time // nmf.timestep)

foreleg_ids = np.zeros(target_num_steps)
middle_stance_ids = np.linspace(swing_periods["RM"][1], 2 * np.pi, target_num_steps)
hind_swing_ids = np.linspace(0.0, swing_periods["RH"][1], target_num_steps)

R_midleg_start = preprogrammed_steps.get_joint_angles("RM", swing_periods["RM"][1])
R_midleg_stretch = np.linspace(
    np.zeros(len(R_midleg_start)), -R_midleg_start, target_num_steps
)

L_midleg_start = preprogrammed_steps.get_joint_angles("LM", swing_periods["LM"][1])
L_midleg_stretch = np.linspace(
    np.zeros(len(L_midleg_start)), -L_midleg_start, target_num_steps
)

adhesion_action = np.array([0.0 if leg.endswith("F") else 1.0 for leg in legs])

all_joint_angles = []

for i in range(target_num_steps):
    joint_angles = []
    for leg in legs:
        if leg.endswith("F"):
            joint_angles.extend(
                preprogrammed_steps.get_joint_angles(leg, foreleg_ids[i])
            )
        elif leg.endswith("M"):
            midleg_joint_angles = preprogrammed_steps.get_joint_angles(
                leg, middle_stance_ids[i]
            )
            if leg.startswith("R"):
                midleg_joint_angles += R_midleg_stretch[i]
            elif leg.startswith("L"):
                midleg_joint_angles += L_midleg_stretch[i]

            joint_angles.extend(midleg_joint_angles)
        else:
            joint_angles.extend(
                preprogrammed_steps.get_joint_angles(leg, hind_swing_ids[i])
            )

    all_joint_angles.append(joint_angles.copy())

    action = {"joints": np.array(joint_angles), "adhesion": adhesion_action}
    nmf.step(action)
    nmf.render()

nmf.save_video("lunging_extend.mp4", 0)

This is of course a very coarse Lunging behaviour, and it would need to be further refined.

# Viewing joint angles and manually actuating neuromechfly

While of course it is possible to engineer joint angles in the way I described just above this might require a strong intuition about what joint do. This can be further obtained by playing with the an xml in an interactive viewer. Here for example we will load and play with the base xml.

In [4]:
from flygym.common import get_data_path

data_path = get_data_path("flygym", "data") / "mjcf/neuromechfly_seqik_kinorder_ypr.xml"

In [5]:
import os

open_viewer_str = f"python3 -m mujoco.viewer --mjcf {data_path}"
os.system(open_viewer_str)

0

Of course if you modified the xml by adding a new actuator or joints you might want to be able to visualize the changes. This can be done by saving the xml to the file.

In [6]:
class AbdomenNMF(NeuroMechFly):
    def _set_joints_stiffness_and_damping(self):
        # Do not forget to call the parent method
        super()._set_joints_stiffness_and_damping()

        # Set the abdomen joints stiffness and damping
        for body_name in ["A1A2", "A3", "A4", "A5", "A6"]:
            body = self.model.find("body", body_name)
            # add pitch degree of freedom to bed the abdomen
            body.add(
                "joint",
                name=f"joint_{body_name}",
                type="hinge",
                pos="0 0 0",
                axis="0 1 0",
                stiffness=5.0,
                springref=0.0,
                damping=5.0,
                dclass="nmf",
            )


abd_nmf = AbdomenNMF(Parameters())

In [7]:
from dm_control.mjcf.export_with_assets import export_with_assets

export_with_assets(abd_nmf.model, "neuromechfly_abdomen_xml")

In [8]:
# get current path
import os

current_path = os.getcwd()

open_viewer_str = f"python3 -m mujoco.viewer --mjcf {current_path}/neuromechfly_abdomen_xml/Animat.xml"
os.system(open_viewer_str)

0